# 문제정의
문제 유형: 회귀 문제 (Regression Problem)
목표: 주어진 제품 및 매장 특성을 기반으로 판매 금액(Item_Outlet_Sales)을 예측한다.
입력 값 (Features): 제품 + 매장 특성
출력 값 (Target): 판매 금액 (연속형 수치)

## 주어진 제품 및 매장 특성들을(X값, 컬럼) 기반으로 판매 금액(Item_Outlet_Sales - 연속형 데이터)를 예측하기

# 1. 라이브러리 및 데이터 불러오기

In [1]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
# train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch4/train.csv")
# test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch4/test.csv")
train

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCR06,12.500,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1,639.1680
1,FDW11,12.600,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1,990.7104
2,FDH32,12.800,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1,2799.6890
3,FDL52,6.635,Regular,0.046351,Frozen Foods,37.4506,OUT017,2007,NaN,Tier 2,Supermarket Type1,1176.4686
4,FDO09,13.500,Regular,0.125170,Snack Foods,261.4910,OUT013,1987,High,Tier 3,Supermarket Type1,3418.8830
...,...,...,...,...,...,...,...,...,...,...,...,...
6813,NCC54,17.750,Low Fat,0.097863,Health and Hygiene,239.3196,OUT049,1999,Medium,Tier 1,Supermarket Type1,3615.2940
6814,NCF19,13.000,Low Fat,0.035252,Household,48.0034,OUT018,2009,Medium,Tier 3,Supermarket Type2,923.4646
6815,FDU11,4.785,Low Fat,0.092576,Breads,121.0098,OUT035,2004,Small,Tier 2,Supermarket Type1,2530.7058
6816,FDK14,6.980,Low Fat,0.041189,Canned,82.8934,OUT045,2002,NaN,Tier 2,Supermarket Type1,818.9340


# 2.탐색적 데이터 분석(EDA)
구조 확인(ex> 이상치, 결측치)

In [3]:
#----
#샘플링 해서 데이터 확인
#---
# train.head()
# train.tail()
# train.sample(5)

#Item_Identifier, Outlet_Identifier와 같은 식별값들은 필요 없을 듯! drop해서 삭제

In [4]:
train.shape, test.shape
#(6818,12) , (1705,11)

# 8:2나 9:1이면 적당함!

((6818, 12), (1705, 11))

In [5]:
train.info() # Item_Outlet_Sales가 y값
             # 타깃값 : 해당 상품이 특정 매장에서 기록한 실제 판매액
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6818 non-null   object 
 1   Item_Weight                5656 non-null   float64
 2   Item_Fat_Content           6818 non-null   object 
 3   Item_Visibility            6818 non-null   float64
 4   Item_Type                  6818 non-null   object 
 5   Item_MRP                   6818 non-null   float64
 6   Outlet_Identifier          6818 non-null   object 
 7   Outlet_Establishment_Year  6818 non-null   int64  
 8   Outlet_Size                4878 non-null   object 
 9   Outlet_Location_Type       6818 non-null   object 
 10  Outlet_Type                6818 non-null   object 
 11  Item_Outlet_Sales          6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIn

In [6]:
# 수치형 데이터 확인
train.describe() # 수치형 데이터의 이상치를 확인하거나 중앙값, 표준편차, 평균 확인
                 # 데이터가 치우쳐져있는 정도 확인 가능

                 # mean(평균)이나 25%(1/4)의 값으로 Item_Weight에 존재하는 결측값
                 # 채우는 것을 추천
test.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
count,1404.000000,1705.000000,1705.000000,1705.000000
mean,12.796984,0.066174,143.285107,1997.615836
std,4.613957,0.052465,63.063146,8.497423
min,4.555000,0.000000,31.955800,1985.000000
25%,8.710000,0.027571,94.743600,1987.000000
50%,12.600000,0.054234,144.512800,1999.000000
75%,16.700000,0.092927,187.755600,2004.000000
max,21.350000,0.311090,266.888400,2009.000000


In [7]:
# 범주형 데이터 확인
# 최빈값을 확인할 때 보통 쓰임
# 이상치에 최빈값을 삽입해보는 것도 방법임

# unique : 중복값
# top : 최다 등록된 것
# freq : 빈도수

train.describe(include='O')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,6818,6818,6818,6818,4878,6818,6818
unique,1554,5,16,10,3,3,4
top,FDW26,Low Fat,Snack Foods,OUT046,Medium,Tier 3,Supermarket Type1
freq,9,4092,963,763,2228,2664,4474


In [8]:
# 결측치 확인
train.isnull().sum()
test.isnull().sum()
# 수치형 결측치 = 평균값, 중앙값 채우고 적으면 삭제
# if 이상치가 범주형이면 최빈값으로 아니면 초기값 부여

Item_Identifier                0
Item_Weight                  301
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                  470
Outlet_Location_Type           0
Outlet_Type                    0
dtype: int64

In [9]:
#총 결측치 확인
train.isnull().sum().sum()
test.isnull().sum().sum()

771

# 3. 데이터 전처리
결측치 처리: Item_Weight, Outlet_Size
범주형 데이터: Label Encoding 적용
불필요한 변수 제거: Item_Identifier

In [11]:
# 결측값 채우기(무게 : 최소값, 사이즈 : 평균값으로 채우기)

In [2]:
train.describe()
# 무게 결측값을 최소값으로 채우기
                      #시리즈로 나온 값을 최소값으로 채움..?
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].min())
                                            #fillna(최소값): 결측값 채울 때 사용하는 함수
train.describe()
# Item_Weight: 5656 -> 6818
# 빈 값(결측값)이 최소값으로 채워짐!


# 사이즈 결측값을 최빈값으로 채우기
train.describe(include='O')
# 최빈값 : 'Outlet_Size'
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])
train.info() #전부 6818건이 됨!




#test도 똑같이!
test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].min())
test['Outlet_Size'] = test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0])
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6818 non-null   object 
 1   Item_Weight                6818 non-null   float64
 2   Item_Fat_Content           6818 non-null   object 
 3   Item_Visibility            6818 non-null   float64
 4   Item_Type                  6818 non-null   object 
 5   Item_MRP                   6818 non-null   float64
 6   Outlet_Identifier          6818 non-null   object 
 7   Outlet_Establishment_Year  6818 non-null   int64  
 8   Outlet_Size                6818 non-null   object 
 9   Outlet_Location_Type       6818 non-null   object 
 10  Outlet_Type                6818 non-null   object 
 11  Item_Outlet_Sales          6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIn

In [3]:
# 컬럼 제거(ID 제거)
          #'컬럼명'
train.drop('Item_Identifier', axis=1, inplace=True) # axis= 방향, 0:행, 1:열
                                                    # inplace=True(기본값 : False)
                                                    # 메서드를 실행하면 원본 데이터프레임(df) 자체가 직접 수정
test.drop('Item_Identifier', axis=1, inplace=True)
train

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,12.500,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1,639.1680
1,12.600,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1,990.7104
2,12.800,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1,2799.6890
3,6.635,Regular,0.046351,Frozen Foods,37.4506,OUT017,2007,Medium,Tier 2,Supermarket Type1,1176.4686
4,13.500,Regular,0.125170,Snack Foods,261.4910,OUT013,1987,High,Tier 3,Supermarket Type1,3418.8830
...,...,...,...,...,...,...,...,...,...,...,...
6813,17.750,Low Fat,0.097863,Health and Hygiene,239.3196,OUT049,1999,Medium,Tier 1,Supermarket Type1,3615.2940
6814,13.000,Low Fat,0.035252,Household,48.0034,OUT018,2009,Medium,Tier 3,Supermarket Type2,923.4646
6815,4.785,Low Fat,0.092576,Breads,121.0098,OUT035,2004,Small,Tier 2,Supermarket Type1,2530.7058
6816,6.980,Low Fat,0.041189,Canned,82.8934,OUT045,2002,Medium,Tier 2,Supermarket Type1,818.9340


In [4]:
# -------------------------------------------
# ★ OLS - 선형회귀에서만 사용할 것 (분류는 X)
# -------------------------------------------
from statsmodels.formula.api import ols

# y 항목컬럼명 저장
y_col = "Item_Outlet_Sales"

# y를 뺀 x값 후보들(수치형, 범주형 다 포함됨)
X_cols = [c for c in train.columns if c != y_col] # 조건식을 만족하는 요소들을 c에 담아
                                                  # 리스트로 빼는 중!
X_cols

# formula 생성
formula = f"{y_col} ~" + " + " .join(X_cols)

# OLS 모델 적용
model = ols(formula, data=train).fit() # ols에 담아서 모델화!

#확인
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.564
Method:                 Least Squares   F-statistic:                     285.9
Date:                Mon, 13 Oct 2025   Prob (F-statistic):               0.00
Time:                        02:10:33   Log-Likelihood:                -57565.
No. Observations:                6818   AIC:                         1.152e+05
Df Residuals:                    6786   BIC:                         1.154e+05
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [5]:
# [★] OLS - Pvalue 값 높은값(0.05 이상)만 제거 (이상이면 y값과 상관관계 낮음)

# PValue 0.05 이상인 컬럼 제거해보기
model.pvalues[model.pvalues > 0.05].index.tolist()

# 항목들이 리스트형태로 가져와짐
# list = model.pvalues[model.pvalues > 0.05].index.tolist()
# list # 리스트 + 반복문으로 하나씩 삭제도 가능!

# 컬럼제거
train = train.drop('Item_Fat_Content', axis=1)
train = train.drop('Item_Type', axis=1)
train = train.drop('Outlet_Identifier', axis=1)


test = test.drop('Item_Fat_Content', axis=1)
test = test.drop('Item_Type', axis=1)
test = test.drop('Outlet_Identifier', axis=1)

In [6]:
# OLS 다중공선성 의심(다중공선성 : 설명 변수들(독립변수, X1, X2..) 사이에 서로 강한 상관관계 존재)
# Cond. No.                 1.44e+19 >> 숫자가 높으면 낮춰줘야함
# ex> X1값의 증가가 X2 때문인지 종속변수 때문인지 알 수 없기 때문에

# 발생 원인
# 범주형 독립변수(X1,X2) 간의 중복된 의미가 있을 때
# 해결 -> 범주형 - 수치형 검정(카이제곱)

# 연속된 수치값(ex 날짜정보)가 포함되어있을 때
# 해결 -> 컬럼 삭제

# 수치형 볌수들간의 변수 범위 차이가 클 때
# 해결 -> 스케일링을 통해 범위를 0-1사이로 잡아주는 작업

# Cond.No
# < 30                     | 문제 없음
# 30 ~ 100                 | 약한 다중공선성 가능성
# 100 ~ 1,000              | 다중공선성 의심
# **>1,000**               | ** 심각한 다중공선성 가능성 있음 **
# **>10* (100,000)**       | **수치적으로 매우 불안정 - 경고 수준 **



# 다중공선성 원인 변수 찾기(VIF) - 다중공선성의 정도를 수치로 측정한 지표
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

X_cols = [c for c in train.columns if c != y_col]

X = train[X_cols].select_dtypes(include=['float', 'int']) # 수치형 X들만 추출
vif = pd.DataFrame()

# 컬럼 2개(Feature, VIF)추가
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])] # shape: (행, 열)
print(vif)

# VIF > 10 → 다중공선성 심함
# VIF > 5 → 주의 필요
# → Outlet_Establishment_Year

# 해결책
# 수치형 변수 스케일링 - StandardScaler 또는 MinMaxScaler
# 다중공선성 높은 변수 제거 - VIF 10 이상인 변수 제거
# Ridge/Lasso 회귀로 대체 - 정규화로 공선성 완화

                     Feature        VIF
0                Item_Weight   5.802119
1            Item_Visibility   2.665509
2                   Item_MRP   6.121111
3  Outlet_Establishment_Year  12.748339


In [7]:
# 컬럼제거
test = test.drop('Outlet_Establishment_Year', axis=1)
train = train.drop('Outlet_Establishment_Year', axis=1)

In [6]:
# [★] vif > 10 컬럼 삭제

train = train.drop('Outlet_Establishment_Year', axis=1)
test = test.drop('Outlet_Establishment_Year', axis=1)

In [8]:
# --------------------------------------------------------
# [★] OLS 수치형 변수 스케일링
# --------------------------------------------------------
from sklearn.preprocessing import StandardScaler

# 1. 수치형 컬럼 자동 선택(object 제외)
num_cols = train.select_dtypes(exclude=['object']).columns
# 2. 스케일러 선언
scaler = StandardScaler()
# 3. 수치형 컬럼만 스케일링
train[num_cols] = scaler.fit_transform(train[num_cols])
# 4. 결과 확인
print("스케일링 완료된 수치형 컬럼들:")
print(num_cols.tolist())
print(train.head())

스케일링 완료된 수치형 컬럼들:
['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales']
   Item_Weight  Item_Visibility  Item_MRP Outlet_Identifier Outlet_Size  \
0     0.198448        -1.155377 -1.572722            OUT013        High   
1     0.217440        -0.338271 -1.289008            OUT013        High   
2     0.255424         0.192206 -0.697329            OUT013        High   
3    -0.915442        -0.384799 -1.659095            OUT017      Medium   
4     0.388369         1.149287  1.950774            OUT013        High   

  Outlet_Location_Type        Outlet_Type  Item_Outlet_Sales  
0               Tier 3  Supermarket Type1          -0.909594  
1               Tier 3  Supermarket Type1          -0.703533  
2               Tier 3  Supermarket Type1           0.356826  
3               Tier 2  Supermarket Type1          -0.594648  
4               Tier 3  Supermarket Type1           0.719776  


In [13]:
# 이상치 - 생략

In [11]:
# y값 분할(타깃 분리) - LIGHTGBM 사용할 때 실행 추천!
target = train.pop('Item_Outlet_Sales')

In [15]:
# 레이블인코딩(원-핫 인코딩)
# 범주형 데이터가 수치형 데이터로 바뀌어져있음(ex> Low Fat : 1, Regular:2)

from sklearn.preprocessing import LabelEncoder
cols = train.columns[train.dtypes == object] # 범주형 -> 수치형
for col in cols:
    le = LabelEncoder()

    #피팅되는 경우가 안 맞는 경우 각 행마다 이동할 때 train,text행을 합치기
    le.fit(pd.concat([train[col],test[col]] ,axis=0))
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
train

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,0.198448,-1.155377,-1.572722,1,0,2,1,-0.909594
1,0.217440,-0.338271,-1.289008,1,0,2,1,-0.703533
2,0.255424,0.192206,-0.697329,1,0,2,1,0.356826
3,-0.915442,-0.384799,-1.659095,2,1,1,1,-0.594648
4,0.388369,1.149287,1.950774,1,0,2,1,0.719776
...,...,...,...,...,...,...,...,...
6813,1.195536,0.617792,1.593535,9,1,0,1,0.834905
6814,0.293408,-0.600825,-1.489062,3,1,2,2,-0.742950
6815,-1.266797,0.514896,-0.312741,6,2,1,1,0.199158
6816,-0.849919,-0.485264,-0.926894,7,1,1,1,-0.804222


,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,12.500,1,0.006760,9,42.8112,1,1987,0,2,1
1,12.600,1,0.048741,1,60.4194,1,1987,0,2,1
2,12.800,1,0.075997,6,97.1410,1,1987,0,2,1
3,6.635,2,0.046351,5,37.4506,2,2007,1,1,1
4,13.500,2,0.125170,13,261.4910,1,1987,0,2,1
...,...,...,...,...,...,...,...,...,...,...
6813,17.750,1,0.097863,8,239.3196,9,1999,1,0,1
6814,13.000,1,0.035252,9,48.0034,3,2009,1,2,2
6815,4.785,1,0.092576,1,121.0098,6,2004,2,1,1
6816,6.980,1,0.041189,3,82.8934,7,2002,1,1,1


# 검증데이터 분할

In [17]:
# 8 : 2 = 학습용(train) : 검증용(val)

In [16]:
# train - val(8,2)
from sklearn.model_selection import train_test_split

                        #y_val : 예측된 결과와 비교될 예정
X_train, X_val, y_train, y_val = train_test_split(
    train, # X값
    target, # Y값
    test_size=0.2, #검증 데이터 나눠질 때 비율
    random_state=0 #난수값 고정으로!(동일한 값)
)
X_train.shape, X_val.shape, y_train.shape, y_val.shape
# 결과 : (5454, 10), (1364, 10), (5454,), (1364,)

NameError: name 'target' is not defined

# 학습 및 평가(MSE, MAE, R2...)

In [19]:
# 모델학습 : LIGHTGBM사용

In [17]:
# LIGHTGBM 학습
# category로 빼서 쓰면 인코디을 할 필요가 없음

import lightgbm as lgb
model = lgb.LGBMRegressor(random_state=0, verbose=-1) #verbose=-1 : 학습 중 로그 메시지 출력 X(조용하게 실행)
model.fit(X_train, y_train)
y_pred_2 = model.predict(X_val)


NameError: name 'X_train' is not defined

In [12]:
# 성능지표확인
from sklearn.metrics import mean_squared_error #MSE(평균 제곱 오차)
from sklearn.metrics import mean_absolute_error #MAE(평균 절대 오차)
from sklearn.metrics import root_mean_squared_error #RSME(제곱 평균 오차 루트)
from sklearn.metrics import r2_score #R2Score(결정계수) 
lgm_mse = mean_squared_error(y_val, y_pred_2)
lgm_mae = mean_absolute_error(y_val, y_pred_2)
lgm_rmse = root_mean_squared_error(y_val, y_pred_2)
lgm_r2 = r2_score(y_val, y_pred_2)



print("MSE : ", lgm_mse)
print("MAE : ", lgm_mae)
print("RMSE : ", lgm_rmse)
print("R2 : ", lgm_r2)

NameError: name 'y_val' is not defined

# -------------------------------------------
# 상관계수 확인(x후보들 - y값) 낮은 컬럼 제거
# -------------------------------------------

In [22]:
# 임계값 조절

TARGET_CORR_TH = 0.03 # 상관계수 임계값(0.03인 컬럼들만 유지할 예정)

# 숫자형 기준으로 상관계수 계산
df_corr = train.copy()
df_corr['__target__'] = y_train # y값이 없는 상태라 y값을 포함시킴

corr_with_target = df_corr.corr(numeric_only=True)['__target__'].drop('__target__')
                                #y값 기준으로 수치형 범주 확인 # 자기자신을 제외하고 나머지 
                                # x와 y와의 관계여부를 확인할 때 사용
                                # target과 나머지 간의 상관관계 확인
corr_with_target

# 유지할 컬럼(= 타깃과의 절대 상관이 임계값 이상인 컬럼만 추출)
keep_features = corr_with_target[corr_with_target.abs() >= TARGET_CORR_TH ] .index.tolist()
                                                # abs() : 음수는 절대값으로 처리
                                                # abs() >= 0.03인 컬럼의 이름을 리스트로 빼냄
# y값을 빼낼 때는 y값에 연관이 있는 컬럼들을 빼내야함
keep_features

['Item_Weight',
 'Item_Visibility',
 'Item_MRP',
 'Outlet_Identifier',
 'Outlet_Establishment_Year',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [23]:
#제거할 컬럼
problem_features = corr_with_target[corr_with_target.abs() < TARGET_CORR_TH ] .index.tolist()
problem_features

['Item_Fat_Content', 'Item_Type']

In [24]:
# ------------------------
# 컬럼 제거
# ------------------------
train.drop('Item_Fat_Content', axis=1, inplace=True)
train.drop('Item_Type', axis=1, inplace=True)

test.drop('Item_Fat_Content', axis=1, inplace=True)
test.drop('Item_Type', axis=1, inplace=True)

In [25]:
# ----------------------------------
# 확인
# ----------------------------------
train.shape, test.shape

((6818, 8), (1705, 8))

In [26]:
# 검증 데이터 분할

In [27]:
from sklearn.model_selection import train_test_split

                        #y_val : 예측된 결과와 비교될 예정
X_train_re, X_val_re, y_train_re, y_val_re = train_test_split(
    train, # X값
    target, # Y값
    test_size=0.2, #검증 데이터 나눠질 때 비율
    random_state=0 #난수값 고정으로!(동일한 값)
)
X_train_re.shape, X_val_re.shape, y_train_re.shape, y_val_re.shape

((5454, 8), (1364, 8), (5454,), (1364,))

In [28]:
# LIGHTGBM 학습
import lightgbm as lgb
model = lgb.LGBMRegressor(random_state=0, verbose=-1) #verbose=-1 : 학습 중 로그 메시지 출력 X(조용하게 실행)
model.fit(X_train_re, y_train_re)
y_pred_2_re = model.predict(X_val_re)

In [29]:
# 성능지표확인
lgm_mse_re = mean_squared_error(y_val_re, y_pred_2_re)
lgm_mae_re = mean_absolute_error(y_val_re, y_pred_2_re)
lgm_rmse_re = root_mean_squared_error(y_val_re, y_pred_2_re)
lgm_r2_re = r2_score(y_val_re, y_pred_2_re)



print("MSE : ", lgm_mse_re)
print("MAE : ", lgm_mae_re)
print("RMSE : ", lgm_rmse_re)
print("R2 : ", lgm_r2_re)
# 값이 낮을수록 좋음!

MSE :  1108334.9408928182
MAE :  736.8110133347378
RMSE :  1052.7748766440136
R2 :  0.5730683504694064
